The Jacobi method and the Power method are numerical algorithms used to find eigenvalues and eigenvectors of a given matrix. Both methods are iterative and can be applied to different types of matrices, but they have distinct approaches and objectives.

In this work, I apply Jacobi and Power Method to Hilbert Matrix

In [4]:
import numpy as np
from numpy import linalg as LA
from numpy import array,identity,diagonal
from math import sqrt
from scipy.linalg import hilbert
from numpy.linalg import norm 

A Hilbert matrix is a specific type of square matrix with entries defined by the formula:

H_ij = 1 / (i+j-1)
 
where i and j are the row and column indices,respectively, starting from 1. So, the element in the ith row and jth column of the Hilbert matrix is the reciprocal of the sum of the indices i and j.

Hilbert matrices are interesting because they are well-known examples of ill-conditioned matrices. The condition number of a matrix measures its sensitivity to changes in the input data. The Hilbert matrix has a very high condition number, which means that small changes in its entries can lead to large changes in its inverse. This makes solving linear systems involving Hilbert matrices challenging and prone to numerical instability.

In [18]:
# Here is an example of a 3x3 Hilbert matrix:
hilbert(3)

array([[1.        , 0.5       , 0.33333333],
       [0.5       , 0.33333333, 0.25      ],
       [0.33333333, 0.25      , 0.2       ]])

In [5]:
hilbert_matrix=hilbert(30)
hilbert_matrix

array([[1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.08333333, 0.07692308, 0.07142857, 0.06666667,
        0.0625    , 0.05882353, 0.05555556, 0.05263158, 0.05      ,
        0.04761905, 0.04545455, 0.04347826, 0.04166667, 0.04      ,
        0.03846154, 0.03703704, 0.03571429, 0.03448276, 0.03333333],
       [0.5       , 0.33333333, 0.25      , 0.2       , 0.16666667,
        0.14285714, 0.125     , 0.11111111, 0.1       , 0.09090909,
        0.08333333, 0.07692308, 0.07142857, 0.06666667, 0.0625    ,
        0.05882353, 0.05555556, 0.05263158, 0.05      , 0.04761905,
        0.04545455, 0.04347826, 0.04166667, 0.04      , 0.03846154,
        0.03703704, 0.03571429, 0.03448276, 0.03333333, 0.03225806],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714,
        0.125     , 0.11111111, 0.1       , 0.09090909, 0.08333333,
        0.07692308, 0.07142857, 0.06666667, 0.

# Jacobi Method

Jacobi Method:

The Jacobi method is an iterative technique used to diagonalize a real symmetric matrix by performing a series of Jacobi rotations. It was proposed by Carl Gustav Jacobi in the 19th century. The main aim of the Jacobi method is to find the eigenvalues and eigenvectors of the input matrix. A real symmetric matrix has orthogonal eigenvectors, which means the matrix can be transformed into a diagonal matrix by performing a series of rotations on the off-diagonal elements.

The Jacobi method starts with an initial estimate of the eigenvalues and eigenvectors (usually an identity matrix for eigenvectors), and then it iteratively performs rotations to make the off-diagonal elements converge to zero. The algorithm iteratively searches for the largest off-diagonal element and then performs a rotation to reduce the value of that element, making it closer to zero. This process is repeated until the off-diagonal elements become small enough, and the matrix becomes diagonal (or close to diagonal). The diagonal elements at the end are the eigenvalues, and the columns of the rotation matrix correspond to the eigenvectors.


In [6]:
#https://www-f9.ijs.si/~kersevan/jacobi.py

In [7]:

from numpy import array,identity,diagonal
from math import sqrt

def jacobi(a): # Jacobi method

    def maxElem(a): # Find largest off-diag. element a[k,l]
        n = len(a)
        aMax = 0.0
        for i in range(n-1):
            for j in range(i+1,n):
                if abs(a[i,j]) >= aMax:
                    aMax = abs(a[i,j])
                    k = i; l = j
        return aMax,k,l

    def rotate(a,p,k,l): # Rotate to make a[k,l] = 0
        n = len(a)
        aDiff = a[l,l] - a[k,k]
        if abs(a[k,l]) < abs(aDiff)*1.0e-9: t = a[k,l]/aDiff
        else:
            phi = aDiff/(2.0*a[k,l])
            t = 1.0/(abs(phi) + sqrt(phi**2 + 1.0))
            if phi < 0.0: t = -t
        c = 1.0/sqrt(t**2 + 1.0); s = t*c
        tau = s/(1.0 + c)
        temp = a[k,l]
        a[k,l] = 0.0
        a[k,k] = a[k,k] - t*temp
        a[l,l] = a[l,l] + t*temp
        for i in range(k):      # Case of i < k
            temp = a[i,k]
            a[i,k] = temp - s*(a[i,l] + tau*temp)
            a[i,l] = a[i,l] + s*(temp - tau*a[i,l])
        for i in range(k+1,l):  # Case of k < i < l
            temp = a[k,i]
            a[k,i] = temp - s*(a[i,l] + tau*a[k,i])
            a[i,l] = a[i,l] + s*(temp - tau*a[i,l])
        for i in range(l+1,n):  # Case of i > l
            temp = a[k,i]
            a[k,i] = temp - s*(a[l,i] + tau*temp)
            a[l,i] = a[l,i] + s*(temp - tau*a[l,i])
        for i in range(n):      # Update transformation matrix
            temp = p[i,k]
            p[i,k] = temp - s*(p[i,l] + tau*p[i,k])
            p[i,l] = p[i,l] + s*(temp - tau*p[i,l])
        
    n = len(a)
    maxRot = 5*(n**2)       # Set limit on number of rotations
    p = identity(n)*1.0     # Initialize transformation matrix
    for i in range(maxRot): # Jacobi rotation loop 
        aMax,k,l = maxElem(a)
        if aMax < 1.0e-9: return diagonal(a),p
        rotate(a,p,k,l)
    print('Jacobi method did not converge')


In [8]:

wj, vj = jacobi(hilbert_matrix)


print ("eigenvalues:\n",wj)
print ("eigenvectors:\n",vj)


eigenvalues:
 [1.98649257e+00 5.72599933e-01 2.08785073e-04 1.54814913e-02
 8.58344047e-09 1.69863091e-06 1.99657535e-05 3.61479257e-11
 2.53628370e-12 1.05645646e-01 8.32421518e-12 2.90723716e-11
 5.20533079e-11 1.19178102e-10 1.20981388e-10 5.57153889e-11
 6.95473634e-12 7.06174726e-11 2.08343198e-11 1.54603870e-11
 3.02578168e-11 8.10245499e-11 1.81593950e-11 1.29250800e-07
 1.34400900e-11 2.02013170e-11 5.51132216e-11 4.54031121e-11
 2.39216120e-11 1.92662077e-03]
eigenvectors:
 [[ 6.10768930e-01 -6.49456867e-01  3.15393437e-02  1.94430924e-01
   3.05114210e-04  3.61120754e-03  1.10787318e-02  3.27795162e-05
  -9.94372794e-07  3.99292008e-01 -1.39010281e-05 -2.22250880e-05
  -2.67890065e-05 -4.28229576e-05 -9.55943556e-06 -3.27181432e-05
  -6.95976256e-06 -2.46372692e-05 -6.72751982e-06  1.55418428e-05
   2.47278989e-05  3.38856905e-05  1.67229389e-05 -1.10109002e-03
   9.74218555e-06 -7.08378594e-06 -2.00026610e-05 -1.67646708e-05
  -1.20221748e-05  8.24486201e-02]
 [ 3.92336487e-

# Power Method

Power Method:


The Power method is an iterative algorithm used to find the dominant (largest magnitude) eigenvalue and its corresponding eigenvector of a square matrix. The dominant eigenvalue has the largest magnitude, and the associated eigenvector points in the direction of the most significant stretching or scaling effect when the matrix is applied iteratively.
The Power method starts with an initial vector and repeatedly multiplies it by the matrix, normalizing the resulting vector at each iteration. As the iterations progress, the resulting vector converges to the dominant eigenvector. Simultaneously, the corresponding eigenvalue can be approximated by the ratio of the elements in the resulting vector in consecutive iterations.


The Power method is relatively simple and computationally efficient, making it useful for finding the dominant eigenvalue and eigenvector of large matrices. However, it is essential to be cautious with its use, as it only converges to the dominant eigenvalue and might not work well for matrices with multiple eigenvalues of similar magnitude or matrices with complex eigenvalue spectra.

In [9]:
def normalize(x):
    fac = abs(x).max()
    x_n = x / x.max()
    return fac, x_n

In [10]:
# initial vector x
x = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])


for i in range(30):
    x = np.dot(hilbert_matrix, x)
    eigval, x = normalize(x)
    
print('Eigenvalue:', eigval)
print('Eigenvector:', x)

Eigenvalue: 1.9864925685391468
Eigenvector: [1.         0.35363364 0.21232703 0.18067609 0.1569882  0.14070066
 0.12822606 0.11830646 0.11019517 0.10922174 0.09778674 0.09279745
 0.08843504 0.08458111 0.08114605 0.0780607  0.07527075 0.07273287
 0.07041206 0.06827969 0.06631213 0.06448962 0.06279556 0.06121587
 0.05973848 0.05835308 0.05705068 0.0558235  0.05466472 0.05362036]


The spectral norm of a matrix A is the largest singular value of A 

In [13]:
jacobi_eig = 1.98649257e+00 # dominant eigenvalue

In [14]:
scipy.linalg.svdvals(vj)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [15]:
np.amax(np.linalg.svd(vj,compute_uv=0),axis=-1) # spectral norm of Jacobi

1.0000000000000004

In [16]:
norm(vj,2) # L2 norm of matrix containing eigenvectors from Jacobi method

1.0000000000000004

In [17]:
norm(x,2) # L2 norm of matrix containing eigenvectors from Power method

1.184733439969541

In summary, both the Jacobi method and the Power method are used for different purposes related to eigenvalue and eigenvector computations. The Jacobi method diagonalizes real symmetric matrices to find all eigenvalues and eigenvectors, while the Power method efficiently approximates the dominant eigenvalue and eigenvector of any square matrix.

## Comparison

Let's compare the L2 norms of the matrices containing eigenvectors obtained from the Jacobi method and the Power method:

L2 norm of eigenvectors from the Jacobi method: 1.0000000000000004

L2 norm of eigenvectors from the Power method: 1.184733439969541

The L2 norm is also known as the Euclidean norm, and it measures the length or magnitude of a vector in a vector space. In this context, it represents how spread out or "far" the eigenvectors are from the origin.

Comparison:

The L2 norm of the eigenvectors from the Jacobi method is approximately 1.0, which means that the eigenvectors obtained from the Jacobi method have a length close to 1. This indicates that the eigenvectors are relatively well-scaled and are not too spread out.

The L2 norm of the eigenvectors from the Power method is approximately 1.1847. This value is larger than 1, which indicates that the eigenvectors obtained from the Power method are more spread out or have larger magnitudes compared to the eigenvectors from the Jacobi method.

In this comparison, a smaller L2 norm implies that the eigenvectors are better normalized, with smaller magnitudes and less spread out. So, in terms of spectral norms, the eigenvectors obtained from the Jacobi method appear to be better normalized than the eigenvectors from the Power method.